In [1]:
import pandas as pd
import numpy as np
import time
import datetime

%run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
%run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

### Carga de datos de entrada

In [2]:
# MOVIMIENTOS

DF = pd.read_csv('../../data/DataFrame_Final_Cierre.csv', parse_dates=['FECHA'])

df_tracks=DF.copy()
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2082289 entries, 0 to 2082288
Data columns (total 20 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             int64         
 1   ANIO                 int64         
 2   MES                  int64         
 3   DIA                  int64         
 4   HORA                 int64         
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           int64         
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   Es_Festivo           int64         
 10  Es_FinSemana         int64         
 11  TEMPERATURA          float64       
 12  VIENTO               float64       
 13  PRESION              int64         
 14  HUMEDAD              int64         
 15  PRECIPITACION_1h     float64       
 16  PRECIPITACION_3h     float64       
 17  DESC_TIEMPO          object        
 18  DESC_TIEMPO_detalle  object        
 19  DEMANDA              

In [3]:
# ESTACIONES: Lista de estaciones en DF

df_estaciones = df_tracks.ESTACION.unique()
df_estaciones

array([  6,   8,   9,  12,  15,  17,  19,  20,  21,  25,  26,  31,  36,
        39,  40,  41,  43,  44,  45,  47,  48,  49,  50,  52,  53,  54,
        55,  56,  57,  58,  59,  63,  65,  66,  67,  69,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  85,  87,  90,  92,  93,
        95,  96,  98, 101, 102, 103, 104, 105, 106, 107, 110, 113, 114,
       122, 123, 124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 139, 145, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158,
       160, 161, 162, 163, 164, 166, 167, 168, 170, 172, 175,   7,  11,
        13,  14,  16,  18,  27,  34,  51,  64,  71,  83,  91,  94, 118,
       140, 141, 142, 143, 147,   5,  42,  46,  62,  86, 117, 121, 137,
       165,   4, 111, 116, 146, 151,  30,  89, 108, 112, 159, 169,   3,
        60,  88,  97, 109, 115, 120, 138, 171,  33,  84,  99, 127, 144,
       173, 174,   1, 100,  10,  35,  38,   2,  32,  37, 119,  61,  29,
        28, 179, 191, 178, 195, 199, 215, 192, 193, 198], dtype=

In [4]:
# Meteorologia

db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Meteo_US_NivelHora')

t_ini = time.time()
#data_Meteo = _consulta_meteoUS_por_anio(db_Meteo, 2020)
data_Meteo = _consulta_meteoUS_full(db_Meteo)
t_end = time.time()

print (t_end - t_ini)

# Se agrega campo FECHA para join
data_Meteo['FECHA'] =pd.to_datetime({'year': data_Meteo['ANIO'],
                                          'month': data_Meteo["MES"]
                                          ,'day':  data_Meteo['DIA']
                                          ,'hour':data_Meteo['HORA']},
                                          format='%d-%m-%Y', errors='coerce')

data_Meteo.info()

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Meteo_US_NivelHora')
1.424354076385498
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ANIO                 26304 non-null  int64         
 1   MES                  26304 non-null  int64         
 2   DIA                  26304 non-null  int64         
 3   HORA                 26304 non-null  int64         
 4   TEMPERATURA          26304 non-null  object        
 5  

### Creación de DF de fechas

In [5]:
# FECHAS DE FESTIVOS

df_festivos_in = pd.read_csv('../../data/Festivos_Madrid.csv', sep=';')
# Se agrega campo FECHA para join
df_festivos_in['FECHA'] =pd.to_datetime({'year': df_festivos_in['ANIO'],
                                          'month': df_festivos_in["MES"]
                                          ,'day':  df_festivos_in['DIA']},
                                          format='%d-%m-%Y', errors='coerce')
df_festivos_in.info()
df_festivos = df_festivos_in[df_festivos_in['FESTIVO']==1]

df_festivos.info()
df_festivos[df_festivos['ANIO']==2018].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ANIO     1461 non-null   int64         
 1   MES      1461 non-null   int64         
 2   DIA      1461 non-null   int64         
 3   FESTIVO  1461 non-null   int64         
 4   FECHA    1461 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 57.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 1454
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ANIO     57 non-null     int64         
 1   MES      57 non-null     int64         
 2   DIA      57 non-null     int64         
 3   FESTIVO  57 non-null     int64         
 4   FECHA    57 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 2.7 KB


,ANIO,MES,DIA,FESTIVO,FECHA
365,2018,1,1,1,2018-01-01
370,2018,1,6,1,2018-01-06
452,2018,3,29,1,2018-03-29
453,2018,3,30,1,2018-03-30
485,2018,5,1,1,2018-05-01


### Creación de rango total de fechas

In [6]:
# Creación de rangos de fecha válidos
# sdate = datetime.strptime('01/01/2018 00', '%d/%m/%Y %H')
# edate = datetime.strptime('31/01/2018 23', '%d/%m/%Y %H')

sdate = np.datetime64('2018-01-01T00:00:00')
edate = np.datetime64('2020-02-29T23:00:00')

df_dates=pd.date_range(start=sdate, end=edate, freq='H')

print(df_dates.shape)
df_dates

(18960,)


DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00', '2018-01-01 03:00:00',
               '2018-01-01 04:00:00', '2018-01-01 05:00:00',
               '2018-01-01 06:00:00', '2018-01-01 07:00:00',
               '2018-01-01 08:00:00', '2018-01-01 09:00:00',
               ...
               '2020-02-29 14:00:00', '2020-02-29 15:00:00',
               '2020-02-29 16:00:00', '2020-02-29 17:00:00',
               '2020-02-29 18:00:00', '2020-02-29 19:00:00',
               '2020-02-29 20:00:00', '2020-02-29 21:00:00',
               '2020-02-29 22:00:00', '2020-02-29 23:00:00'],
              dtype='datetime64[ns]', length=18960, freq='H')

### EJECUCION DE PROCESO 

In [ ]:
#df_estaciones = [175,45,15]
#df_estaciones

In [7]:
i=0
df_final = pd.DataFrame(columns = ['ESTACION','ANIO','MES','DIA','HORA','FECHA','DIA_SEMANA','AM_PM','TEMPORADA',
                                   'Es_Festivo','Es_FinSemana','TEMPERATURA','VIENTO','PRESION','HUMEDAD',
                                   'PRECIPITACION_1h','PRECIPITACION_3h','DESC_TIEMPO','DESC_TIEMPO_detalle','DEMANDA'])

t_ini = time.time()

for estacion in df_estaciones:
    print('Estacion: '+str(estacion))
    
    df_data = df_tracks[df_tracks['ESTACION']==estacion]
    
    df_final_estacion = pd.DataFrame(columns = ['ESTACION','ANIO','MES','DIA','HORA','FECHA','DIA_SEMANA','AM_PM','TEMPORADA',
                                   'Es_Festivo','Es_FinSemana','TEMPERATURA','VIENTO','PRESION','HUMEDAD',
                                   'PRECIPITACION_1h','PRECIPITACION_3h','DESC_TIEMPO','DESC_TIEMPO_detalle','DEMANDA'])    
    
    t_ini_estacion = time.time()
    for fecha in df_dates:
        
        # Variable dia semana
        if fecha.isoweekday()==7:
            dia_semana=1
        else:
            dia_semana=fecha.isoweekday()+1
            
        # Variable Es_Festivo
        fec_festivo = np.datetime64(fecha.date())        
        if fec_festivo in df_festivos['FECHA'].values:
            es_festivo = 1
        else:
            es_festivo = 0
                        
        # Variable Es_FinSemana
        if ((dia_semana==1) or (dia_semana==7)):
            es_finsemana = 1
        else:
            es_finsemana = 0
        
        # Variable AM_PM
        if fecha.hour<=12:
            ampm='AM'
        else:
            ampm='PM'
            
        # Variable TEMPORADA
        if fecha.month<=3:
            temporada='INVIERNO'
        elif fecha.month<=6:
            temporada='PRIMAVERA'
        elif fecha.month<=9:
            temporada='VERANO'
        else:
            temporada='OTOÑO'
                
                
        if fecha not in df_data[df_data['ESTACION']==estacion]['FECHA'].values:
            i=i+1
            meteo = data_Meteo[data_Meteo['FECHA']==fecha]
            new_row_dict = {
                'ESTACION': estacion,
                'ANIO': fecha.year,
                'MES': fecha.month,
                'DIA': fecha.day,
                'HORA': fecha.hour,
                'FECHA': fecha,
                'DIA_SEMANA': dia_semana,
                'AM_PM': ampm,
                'TEMPORADA': temporada,
                'Es_Festivo': es_festivo,
                'Es_FinSemana': es_finsemana,
                'TEMPERATURA': float(meteo.TEMPERATURA.iloc[0]),
                'VIENTO': float(meteo.VIENTO.iloc[0]),
                'PRESION': float(meteo.PRESION.iloc[0]),
                'HUMEDAD': float(meteo.HUMEDAD.iloc[0]),
                'PRECIPITACION_1h': float(meteo.PRECIPITACION_1h.iloc[0]),
                'PRECIPITACION_3h': float(meteo.PRECIPITACION_3h.iloc[0]),
                'DESC_TIEMPO': meteo.DESC_TIEMPO.iloc[0],
                'DESC_TIEMPO_detalle': meteo.DESC_TIEMPO_detalle.iloc[0],
                'DEMANDA': 0
            }
            
            new_row = pd.DataFrame(new_row_dict, index=[0])
            
            #print(meteo.TEMPERATURA.iloc[0])
            df_final_estacion = pd.concat([df_final_estacion, new_row])
        else:
            #print(df_data[(df_data['ESTACION']==estacion) & (df_data['FECHA']==fecha)])
            old_row = df_data[(df_data['ESTACION']==estacion) & (df_data['FECHA']==fecha)]
            df_final_estacion = pd.concat([df_final_estacion, old_row])
            
            
    df_final = pd.concat([df_final, df_final_estacion])
    
    t_end_estacion = time.time()
    print('Fechas vacías:' + str(i))
    print('Tiempo: '+str(t_end_estacion-t_ini_estacion))
    i=0
    

t_end = time.time()
print (t_end - t_ini)

Estacion: 6
Fechas vacías:3970
Tiempo: 46.515997648239136
Estacion: 8
Fechas vacías:4992
Tiempo: 46.088478803634644
Estacion: 9
Fechas vacías:4752
Tiempo: 45.745094776153564
Estacion: 12
Fechas vacías:5697
Tiempo: 45.12749171257019
Estacion: 15
Fechas vacías:7969
Tiempo: 42.315261363983154
Estacion: 17
Fechas vacías:5148
Tiempo: 45.26795721054077
Estacion: 19
Fechas vacías:3740
Tiempo: 45.75401449203491
Estacion: 20
Fechas vacías:6726
Tiempo: 45.39063119888306
Estacion: 21
Fechas vacías:6687
Tiempo: 45.22220468521118
Estacion: 25
Fechas vacías:6345
Tiempo: 46.16449189186096
Estacion: 26
Fechas vacías:4013
Tiempo: 45.62113308906555
Estacion: 31
Fechas vacías:4463
Tiempo: 45.2908456325531
Estacion: 36
Fechas vacías:7719
Tiempo: 42.04516887664795
Estacion: 39
Fechas vacías:7973
Tiempo: 42.013205766677856
Estacion: 40
Fechas vacías:7688
Tiempo: 41.96491885185242
Estacion: 41
Fechas vacías:4824
Tiempo: 51.00578212738037
Estacion: 43
Fechas vacías:3715
Tiempo: 57.091110706329346
Estacion: 44

Fechas vacías:9884
Tiempo: 39.31080651283264
Estacion: 169
Fechas vacías:4343
Tiempo: 42.96357846260071
Estacion: 3
Fechas vacías:4887
Tiempo: 43.49853038787842
Estacion: 60
Fechas vacías:7195
Tiempo: 40.00242042541504
Estacion: 88
Fechas vacías:11666
Tiempo: 39.704761266708374
Estacion: 97
Fechas vacías:9542
Tiempo: 39.600208044052124
Estacion: 109
Fechas vacías:9576
Tiempo: 39.602285861968994
Estacion: 115
Fechas vacías:7231
Tiempo: 39.58305096626282
Estacion: 120
Fechas vacías:10404
Tiempo: 39.68964219093323
Estacion: 138
Fechas vacías:9641
Tiempo: 39.906845569610596
Estacion: 171
Fechas vacías:6684
Tiempo: 42.996474742889404
Estacion: 33
Fechas vacías:7165
Tiempo: 39.422194957733154
Estacion: 84
Fechas vacías:5256
Tiempo: 43.13354849815369
Estacion: 99
Fechas vacías:6710
Tiempo: 42.93914437294006
Estacion: 127
Fechas vacías:8639
Tiempo: 39.56874203681946
Estacion: 144
Fechas vacías:11484
Tiempo: 39.54204273223877
Estacion: 173
Fechas vacías:11253
Tiempo: 39.5763099193573
Estacion: 

In [8]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3393840 entries, 0 to 2082287
Data columns (total 20 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ESTACION             object        
 1   ANIO                 object        
 2   MES                  object        
 3   DIA                  object        
 4   HORA                 object        
 5   FECHA                datetime64[ns]
 6   DIA_SEMANA           object        
 7   AM_PM                object        
 8   TEMPORADA            object        
 9   Es_Festivo           object        
 10  Es_FinSemana         object        
 11  TEMPERATURA          float64       
 12  VIENTO               float64       
 13  PRESION              object        
 14  HUMEDAD              object        
 15  PRECIPITACION_1h     float64       
 16  PRECIPITACION_3h     float64       
 17  DESC_TIEMPO          object        
 18  DESC_TIEMPO_detalle  object        
 19  DEMANDA              

In [9]:
df_final

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,Es_Festivo,Es_FinSemana,TEMPERATURA,VIENTO,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,DEMANDA
0,6,2018,1,1,0,2018-01-01 00:00:00,2,AM,INVIERNO,1,0,6.63,5.70,1031,56,0.0,0.0,Clear,sky is clear,2
0,6,2018,1,1,1,2018-01-01 01:00:00,2,AM,INVIERNO,1,0,6.47,5.70,1031.0,55.0,0.0,0.0,Clear,sky is clear,0
184,6,2018,1,1,2,2018-01-01 02:00:00,2,AM,INVIERNO,1,0,5.58,4.90,1031,60,0.0,0.0,Clear,sky is clear,2
0,6,2018,1,1,3,2018-01-01 03:00:00,2,AM,INVIERNO,1,0,6.58,7.20,1030.0,59.0,0.0,0.0,Clear,sky is clear,0
305,6,2018,1,1,4,2018-01-01 04:00:00,2,AM,INVIERNO,1,0,5.14,4.63,1032,57,0.0,0.0,Clear,sky is clear,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,198,2020,2,29,19,2020-02-29 19:00:00,7,PM,INVIERNO,0,1,12.17,8.80,1013.0,60.0,0.0,0.0,Clouds,few clouds,0
0,198,2020,2,29,20,2020-02-29 20:00:00,7,PM,INVIERNO,0,1,11.49,8.80,1014.0,61.0,0.0,0.0,Clouds,few clouds,0
0,198,2020,2,29,21,2020-02-29 21:00:00,7,PM,INVIERNO,0,1,11.11,7.20,1014.0,67.0,0.0,0.0,Clouds,few clouds,0
0,198,2020,2,29,22,2020-02-29 22:00:00,7,PM,INVIERNO,0,1,10.31,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,0


### Validación de resultados en base a suma de DEMANDA

In [24]:
pd.set_option('display.max_rows', None)

In [25]:
# DF Original
df_tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))
#df_tracks.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count')).to_clipboard()

,DEMANDA,CUENTA
ESTACION,,
1,61287,13464
2,26868,9225
3,50729,14073
4,34466,11460
5,31397,12517
6,64138,14990
7,38392,12999
8,43817,13968
9,73651,14208


In [26]:
# DF Corregido
df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count'))
#df_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'sum'), CUENTA=('DEMANDA', 'count')).to_clipboard()

,DEMANDA,CUENTA
ESTACION,,
1,61287,18960
2,26868,18960
3,50729,18960
4,34466,18960
5,31397,18960
6,64138,18960
7,38392,18960
8,43817,18960
9,73651,18960


In [27]:
pd.set_option('display.max_rows', 10)

### Export df_final a CSV

In [28]:
df_final.to_csv('../../data/DataFrame_Final_Cierre_FullFecha.csv', index=False)